In [1]:
import pandas as pd
import numpy as np

In [2]:
TEAMS = {"MIBR": 0, 
         "Leviatan": 1, 
         "Sentinels": 2, 
         "NRG Esports": 3, 
         "FURIA": 4, 
         "100 Thieves": 5, 
         "LOUD" : 6, 
         "Evil Geniuses" : 7, 
         "G2 Esports" : 8, 
         "Cloud9" : 9, 
         "KRÃœ Esports" : 10, 
         "Titan Esports Club" : 11, 
         "JDG Esports" : 12, 
         "All Gamers" : 13, 
         "TYLOO" : 14, 
         "Bilibili Gaming" : 15, 
         "Wolves Esports" : 16, 
         "Dragon Ranger Gaming" : 17, 
         "Nova Esports" : 18, 
         "FunPlus Phoenix" : 19, 
         "Trace Esports" : 20, 
         "EDward Gaming" : 21, 
         "ZETA DIVISION" : 22, 
         "DRX" : 23, 
         "Team Secret" : 24, 
         "BLEED" : 25, 
         "T1" : 26, 
         "Gen.G" : 27, 
         "Paper Rex" : 28, 
         "Talon Esports" : 29, 
         "Rex Regum Qeon" : 30, 
         "DetonatioN FocusMe" : 31, 
         "Global Esports" : 32, 
         "FUT Esports" : 33, 
         "KOI" : 34, 
         "BBL Esports" : 35, 
         "FNATIC" : 36, 
         "Team Heretics" : 37, 
         "Natus Vincere" : 38, 
         "GIANTX" : 39, 
         "Gentle Mates" : 40, 
         "Team Vitality" : 41, 
         "Team Liquid" : 42, 
         "Karmine Corp": 43}

MAPS = {"Bind" : 0, 
        "Haven" : 1, 
        "Split" : 2, 
        "Ascent" : 3,
        "Icebox" : 4, 
        "Breeze" : 5, 
        "Fracture" : 6, 
        "Abyss" : 7, 
        "Lotus" : 8, 
        "Sunset" : 9, 
        "Pearl" : 10}

In [72]:
data = pd.read_csv("Dataset/maps_scores.csv")
data = data[["Map", "Team A", "Team B", "Team A Score", "Team B Score"]]
data["Team A Delta"] = data["Team A Score"] - data["Team B Score"]
data["Team B Delta"] = data["Team B Score"] - data["Team A Score"]
data.loc[data["Team A Delta"] < 0, "Team A Win Chance"] = 0
data.loc[data["Team A Delta"] > 0, "Team A Win Chance"] = 100
data

,Map,Team A,Team B,Team A Score,Team B Score,Team A Delta,Team B Delta,Team A Win Chance
0,Ascent,MIBR,Leviatán,9,13,-4,4,0.0
1,Icebox,MIBR,Leviatán,7,13,-6,6,0.0
2,Lotus,Sentinels,NRG Esports,13,8,5,-5,100.0
3,Sunset,Sentinels,NRG Esports,14,12,2,-2,100.0
4,Icebox,FURIA,100 Thieves,14,12,2,-2,100.0
...,...,...,...,...,...,...,...,...
1013,Lotus,DRX,Gen.G,8,13,-5,5,0.0
1014,Ascent,Paper Rex,Gen.G,5,13,-8,8,0.0
1015,Split,Paper Rex,Gen.G,13,4,9,-9,100.0
1016,Lotus,Paper Rex,Gen.G,11,13,-2,2,0.0


In [71]:
ACS = pd.read_csv("Dataset/overview.csv")
ACS = ACS[["Match Type", "Match Name", "Team", "Average Combat Score"]]
ACS.groupby("Team")
ACS

,Match Type,Match Name,Team,Average Combat Score
0,Week 1,MIBR vs Leviatán,MIBR,251
1,Week 1,MIBR vs Leviatán,MIBR,224
2,Week 1,MIBR vs Leviatán,MIBR,170
3,Week 1,MIBR vs Leviatán,MIBR,159
4,Week 1,MIBR vs Leviatán,MIBR,148
...,...,...,...,...
10175,Grand Final,Paper Rex vs Gen.G,Gen.G,291
10176,Grand Final,Paper Rex vs Gen.G,Gen.G,269
10177,Grand Final,Paper Rex vs Gen.G,Gen.G,217
10178,Grand Final,Paper Rex vs Gen.G,Gen.G,196
